In [3]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

In [5]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import json
from dataclasses import asdict
import sys
sys.path.insert(0, str(project_root))

# Import the simulator
from simulator.simulator_sensor_network_skewt_dynamic import (
    GridConfig,
    DynConfig,
    MeasConfig,
    SimConfig,
    simulate_many,
    save_npz
)


print("✓ Imported simulator from simulator_sensor_network_skewt_dynamic.py")

✓ Imported simulator from simulator_sensor_network_skewt_dynamic.py


In [9]:
grid = GridConfig(d=144, alpha0=1.0, alpha1=1e-3, beta=8.0)
dyn = DynConfig(alpha=0.9, nu=8.0, gamma_scale=0.1, gamma_vec=None, clip_x=(-10.0, 10.0), seed=42)
meas = MeasConfig(m1=1.0, m2=1.0/3.0)
sim = SimConfig(T=10, n_trials=1, save_lambda=True)

data = simulate_many(grid, dyn, meas, sim)

# Save the simulation data
data_dir = project_root / "simulator" / "data"
data_dir.mkdir(exist_ok=True)

output_path = data_dir / "snlg_skew_dataset.npz"
config_path = data_dir / "snlg_skew_config.json"

# Save arrays
save_npz(str(output_path), data)

# Save configuration as JSON
config = {
    "grid": asdict(grid),
    "dyn": asdict(dyn),
    "meas": asdict(meas),
    "sim": asdict(sim)
}

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✓ Data saved to: {output_path}")
print(f"✓ Config saved to: {config_path}")


✓ Data saved to: /Users/amber_test/Desktop/filter/simulator/data/snlg_skew_dataset.npz
✓ Config saved to: /Users/amber_test/Desktop/filter/simulator/data/snlg_skew_config.json


# Filter Comparison Experiments

Now we'll run multiple filters on the simulated skewed-t data over 100 simulation trials and compare their performance:
- **EDH**: 200 particles and 10,000 particles
- **LEDH**: 200 particles
- **EKF**: Extended Kalman Filter
- **UKF**: Unscented Kalman Filter

**Experiments:**
- Two grid dimensions: d=144 and d=400
- Metrics tracked: Average MSE, Average ESS (for particle filters), Execution time

In [10]:
# Import filter implementations
from models.unscented_kalman_filter import UnscentedKalmanFilter, UKFState
from models.extended_kalman_filter import ExtendedKalmanFilter, EKFState
from models.EDH_particle_filter import EDHFlowPF, EDHConfig, EKFTracker, UKFTracker, PFState
from models.LEDH_particle_filter import LEDHFlowPF, LEDHConfig
from models.LEDH_particle_filter import PFState as LEDHPFState
import time
import pandas as pd

print("✓ Imported filter implementations")

✓ Imported filter implementations


In [11]:
# Helper function to prepare model functions for filters
def prepare_skewt_model(grid_cfg, dyn_cfg, meas_cfg, Sigma, L, gamma):
    """Prepare dynamics and measurement functions for filters."""
    d = grid_cfg.d
    alpha = dyn_cfg.alpha
    nu = dyn_cfg.nu
    m1, m2 = meas_cfg.m1, meas_cfg.m2
    
    # Process model: x_k = alpha * x_{k-1} + noise
    def g(x, u=None, v=None):
        """Simplified linear dynamics for EKF/UKF (ignoring skew for now)."""
        return alpha * x
    
    # Measurement model: z ~ Poisson(m1 * exp(m2 * x))
    def h(x):
        """Expected measurement (linearization point)."""
        return m1 * np.exp(m2 * x)
    
    # Jacobian of h
    def jacobian_h(x):
        """Jacobian of measurement model."""
        return np.diag(m1 * m2 * np.exp(m2 * x))
    
    # Log transition density (approximation for skewed-t)
    def log_trans_pdf(x_k, x_prev):
        """Log p(x_k | x_{k-1}) - Gaussian approximation."""
        mean = alpha * x_prev
        diff = x_k - mean
        return -0.5 * np.dot(diff, np.linalg.solve(Sigma, diff))
    
    # Log likelihood (Poisson)
    def log_like_pdf(z_k, x_k):
        """Log p(z_k | x_k) for Poisson observations."""
        lam = m1 * np.exp(m2 * x_k)
        lam = np.clip(lam, 1e-10, 1e10)  # Numerical stability
        return np.sum(z_k * np.log(lam) - lam - np.log(np.maximum(1, np.arange(1, len(z_k)+1))))
    
    # Process and measurement noise covariances
    Q = Sigma.copy()  # Approximation
    R = np.diag(m1 * np.exp(m2 * np.zeros(d)))  # Linearized around zero
    
    return g, h, jacobian_h, log_trans_pdf, log_like_pdf, Q, R

def compute_mse(x_true, x_est):
    """Compute Mean Squared Error."""
    return np.mean((x_true - x_est) ** 2)

print("✓ Helper functions defined")

✓ Helper functions defined


In [18]:
# Function to run all filters on one dimension setting
def run_filter_experiments(d_val, n_trials=1):
    """Run all filter experiments for a given dimension."""
    print(f"\n{'='*70}")
    print(f"Running experiments for d={d_val}")
    print(f"{'='*70}\n")
    
    # Configure simulation for this dimension
    grid_cfg = GridConfig(d=d_val, alpha0=1.0, alpha1=1e-3, beta=8.0)
    dyn_cfg = DynConfig(alpha=0.9, nu=8.0, gamma_scale=0.1, gamma_vec=None, 
                        clip_x=(-10.0, 10.0), seed=42)
    meas_cfg = MeasConfig(m1=1.0, m2=1.0/3.0)
    sim_cfg = SimConfig(T=10, n_trials=n_trials, save_lambda=True)
    
    # Generate data
    print(f"Simulating {n_trials} trial(s)...")
    data = simulate_many(grid_cfg, dyn_cfg, meas_cfg, sim_cfg)
    
    # Extract arrays - NOTE: simulate_many returns Sigma_list, L_list, gamma_list
    X_all = data['X']  # (n_trials, T, d)  <-- Note: T not T+1 for skewt simulator!
    Z_all = data['Z']  # (n_trials, T, d)
    Sigma = data['Sigma_list'][0]  # (d, d) - use first trial's Sigma (all same)
    L = data['L_list'][0]  # (d, d) - use first trial's L
    gamma = data['gamma_list'][0]  # (d,) - use first trial's gamma
    
    print(f"✓ Data generated: X shape={X_all.shape}, Z shape={Z_all.shape}")
    
    # Prepare model functions
    g, h, jacobian_h, log_trans_pdf, log_like_pdf, Q, R = prepare_skewt_model(
        grid_cfg, dyn_cfg, meas_cfg, Sigma, L, gamma
    )
    
    # Storage for results
    results = []
    
    # Run filters on each trial
    for trial_idx in range(n_trials):
        print(f"\n--- Trial {trial_idx + 1}/{n_trials} ---")
        
        X_true = X_all[trial_idx]  # (T, d)  <-- States at times 1..T
        Z_obs = Z_all[trial_idx]   # (T, d)  <-- Observations at times 1..T
        T = Z_obs.shape[0]
        
        # Initial state
        x0 = np.zeros(d_val)
        P0 = Sigma.copy()
        
        # === 1. EKF ===
        print("Running EKF...")
        ekf = ExtendedKalmanFilter(g, h, Q, R, jac_g=None, jac_h=jacobian_h)
        ekf_state = EKFState(mean=x0.copy(), cov=P0.copy(), t=0)
        
        start_time = time.time()
        ekf_estimates = []
        for t in range(T):
            ekf_state = ekf.predict(ekf_state, u=None)
            ekf_state = ekf.update(ekf_state, Z_obs[t])
            ekf_estimates.append(ekf_state.mean.copy())
        ekf_time = time.time() - start_time
        
        ekf_estimates = np.array(ekf_estimates)  # (T, d)
        ekf_mse = np.mean([compute_mse(X_true[t], ekf_estimates[t]) for t in range(T)])
        
        results.append({
            'dimension': d_val,
            'trial': trial_idx + 1,
            'filter': 'EKF',
            'n_particles': None,
            'avg_mse': ekf_mse,
            'avg_ess': None,
            'exec_time': ekf_time
        })
        print(f"  EKF - MSE: {ekf_mse:.6f}, Time: {ekf_time:.4f}s")
        
        # === 2. UKF ===
        print("Running UKF...")
        ukf = UnscentedKalmanFilter(g, h, Q, R, alpha=1e-3, beta=2.0, kappa=0.0)
        ukf_state = UKFState(mean=x0.copy(), cov=P0.copy(), t=0)
        
        start_time = time.time()
        ukf_estimates = []
        for t in range(T):
            ukf_state = ukf.predict(ukf_state, u=None)
            ukf_state = ukf.update(ukf_state, Z_obs[t])
            ukf_estimates.append(ukf_state.mean.copy())
        ukf_time = time.time() - start_time
        
        ukf_estimates = np.array(ukf_estimates)  # (T, d)
        ukf_mse = np.mean([compute_mse(X_true[t], ukf_estimates[t]) for t in range(T)])
        
        results.append({
            'dimension': d_val,
            'trial': trial_idx + 1,
            'filter': 'UKF',
            'n_particles': None,
            'avg_mse': ukf_mse,
            'avg_ess': None,
            'exec_time': ukf_time
        })
        print(f"  UKF - MSE: {ukf_mse:.6f}, Time: {ukf_time:.4f}s")
        
        # Helper function to run particle filters
        def run_pf(filter_name, pf_class, config_class, n_particles):
            print(f"Running {filter_name} ({n_particles} particles)...")
            
            # Create tracker (UKF-based)
            ukf_tracker = UnscentedKalmanFilter(g, h, Q, R, alpha=1e-3, beta=2.0, kappa=0.0)
            ukf_init = UKFState(mean=x0.copy(), cov=P0.copy(), t=0)
            tracker = UKFTracker(ukf_tracker, ukf_init)
            
            # Create particle filter config
            pf_config = config_class(
                n_particles=n_particles,
                n_lambda_steps=8,
                resample_ess_ratio=0.5,
                rng=np.random.default_rng(42)
            )
            
            # Initialize particle filter - NOTE: Only pass R, not Q!
            # Signature: (tracker, g, h, jacobian_h, log_trans_pdf, log_like_pdf, R, config)
            pf = pf_class(tracker, g, h, jacobian_h, log_trans_pdf, log_like_pdf, 
                         R, pf_config)
            
            # Initial particles
            particles_init = x0[:, None] + L @ np.random.randn(d_val, n_particles)
            weights_init = np.ones(n_particles) / n_particles
            pf_state = PFState(
                particles=particles_init.T,  # (N, d)
                weights=weights_init,
                mean=x0.copy(),
                cov=P0.copy()
            )
            
            start_time = time.time()
            pf_estimates = []
            ess_values = []
            
            for t in range(T):
                pf_state = pf.step(pf_state, Z_obs[t])
                pf_estimates.append(pf_state.mean.copy())
                
                # Calculate ESS
                from models.EDH_particle_filter import effective_sample_size
                ess = effective_sample_size(pf_state.weights)
                ess_values.append(ess)
            
            pf_time = time.time() - start_time
            
            pf_estimates = np.array(pf_estimates)  # (T, d)
            pf_mse = np.mean([compute_mse(X_true[t], pf_estimates[t]) for t in range(T)])
            avg_ess = np.mean(ess_values)
            
            results.append({
                'dimension': d_val,
                'trial': trial_idx + 1,
                'filter': filter_name,
                'n_particles': n_particles,
                'avg_mse': pf_mse,
                'avg_ess': avg_ess,
                'exec_time': pf_time
            })
            print(f"  {filter_name}({n_particles}) - MSE: {pf_mse:.6f}, ESS: {avg_ess:.2f}, Time: {pf_time:.4f}s")
        
        # === 3. LEDH (200 particles) ===
        run_pf('LEDH', LEDHFlowPF, LEDHConfig, 200)
        
        # === 4. EDH (200 particles) ===
        run_pf('EDH', EDHFlowPF, EDHConfig, 200)
        
        # === 5. EDH (10000 particles) ===
        run_pf('EDH', EDHFlowPF, EDHConfig, 10000)
    
    return results

print("✓ Experiment runner defined")

✓ Experiment runner defined


In [21]:
# Run experiments for d=144 with 100 trials
results_144 = run_filter_experiments(d_val=144, n_trials=100)


Running experiments for d=144

Simulating 100 trial(s)...
✓ Data generated: X shape=(100, 10, 144), Z shape=(100, 10, 144)

--- Trial 1/100 ---
Running EKF...
  EKF - MSE: 0.889947, Time: 0.0175s
Running UKF...
  UKF - MSE: 0.855245, Time: 0.3369s
Running LEDH (200 particles)...
  UKF - MSE: 0.855245, Time: 0.3369s
Running LEDH (200 particles)...
  LEDH(200) - MSE: 0.833806, ESS: 158.31, Time: 13.0236s
Running EDH (200 particles)...
  LEDH(200) - MSE: 0.833806, ESS: 158.31, Time: 13.0236s
Running EDH (200 particles)...
  EDH(200) - MSE: 0.868917, ESS: 149.97, Time: 1.0854s
Running EDH (10000 particles)...
  EDH(200) - MSE: 0.868917, ESS: 149.97, Time: 1.0854s
Running EDH (10000 particles)...
  EDH(10000) - MSE: 0.856478, ESS: 8834.64, Time: 31.2080s

--- Trial 2/100 ---
Running EKF...
  EKF - MSE: 0.883696, Time: 0.0101s
Running UKF...
  EDH(10000) - MSE: 0.856478, ESS: 8834.64, Time: 31.2080s

--- Trial 2/100 ---
Running EKF...
  EKF - MSE: 0.883696, Time: 0.0101s
Running UKF...
  UK

In [22]:
# Run experiments for d=400 with 100 trials
results_400 = run_filter_experiments(d_val=400, n_trials=100)


Running experiments for d=400

Simulating 100 trial(s)...
✓ Data generated: X shape=(100, 10, 400), Z shape=(100, 10, 400)

--- Trial 1/100 ---
Running EKF...
  EKF - MSE: 1.415185, Time: 0.0689s
Running UKF...
✓ Data generated: X shape=(100, 10, 400), Z shape=(100, 10, 400)

--- Trial 1/100 ---
Running EKF...
  EKF - MSE: 1.415185, Time: 0.0689s
Running UKF...
  UKF - MSE: 1.353121, Time: 3.7636s
Running LEDH (200 particles)...
  UKF - MSE: 1.353121, Time: 3.7636s
Running LEDH (200 particles)...
  LEDH(200) - MSE: 1.370166, ESS: 167.49, Time: 88.1368s
Running EDH (200 particles)...
  LEDH(200) - MSE: 1.370166, ESS: 167.49, Time: 88.1368s
Running EDH (200 particles)...
  EDH(200) - MSE: 1.519944, ESS: 170.95, Time: 7.4479s
Running EDH (10000 particles)...
  EDH(200) - MSE: 1.519944, ESS: 170.95, Time: 7.4479s
Running EDH (10000 particles)...
  EDH(10000) - MSE: 1.423793, ESS: 8470.96, Time: 163.2207s

--- Trial 2/100 ---
Running EKF...
  EKF - MSE: 1.181726, Time: 0.0641s
Running UKF.

In [23]:
# Combine results and display as DataFrame
all_results = results_144 + results_400
df_results = pd.DataFrame(all_results)

print("\n" + "="*80)
print("FILTER COMPARISON RESULTS - SKEWED-T SPATIAL NETWORK (100 TRIALS)")
print("="*80)

# Compute average metrics across all trials for each dimension and filter
print("\n" + "="*80)
print("AVERAGE RESULTS OVER 100 TRIALS")
print("="*80)
summary = df_results.groupby(['dimension', 'filter', 'n_particles']).agg({
    'avg_mse': ['mean', 'std'],
    'avg_ess': ['mean', 'std'],
    'exec_time': ['mean', 'std']
}).round(6)

# Flatten column names
summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
summary = summary.reset_index()

print(summary.to_string(index=False))

# Save detailed results (all trials)
results_path = data_dir / "skewt_filter_comparison_results_100trials.csv"
df_results.to_csv(results_path, index=False)
print(f"\n✓ Detailed results saved to: {results_path}")

# Save summary statistics
summary_path = data_dir / "skewt_filter_comparison_summary_100trials.csv"
summary.to_csv(summary_path, index=False)
print(f"✓ Summary statistics saved to: {summary_path}")

# Display clean summary table
print("\n" + "="*80)
print("SUMMARY TABLE (Mean ± Std)")
print("="*80)
for d_val in [144, 400]:
    print(f"\n--- Dimension d={d_val} ---")
    df_d = summary[summary['dimension'] == d_val]
    for _, row in df_d.iterrows():
        filter_name = row['filter']
        n_part = row['n_particles']
        mse_mean = row['avg_mse_mean']
        mse_std = row['avg_mse_std']
        ess_mean = row['avg_ess_mean']
        ess_std = row['avg_ess_std']
        time_mean = row['exec_time_mean']
        time_std = row['exec_time_std']
        
        if pd.notna(n_part):
            print(f"{filter_name}({int(n_part):5d}): MSE={mse_mean:.6f}±{mse_std:.6f}, ESS={ess_mean:6.2f}±{ess_std:5.2f}, Time={time_mean:7.4f}±{time_std:6.4f}s")
        else:
            print(f"{filter_name:12s}: MSE={mse_mean:.6f}±{mse_std:.6f}, ESS={'N/A':>12s}, Time={time_mean:7.4f}±{time_std:6.4f}s")


FILTER COMPARISON RESULTS - SKEWED-T SPATIAL NETWORK (100 TRIALS)

AVERAGE RESULTS OVER 100 TRIALS
 dimension filter  n_particles  avg_mse_mean  avg_mse_std  avg_ess_mean  avg_ess_std  exec_time_mean  exec_time_std
       144    EDH        200.0      1.053517     0.324429    165.200491    12.638366        1.053865       0.056352
       144    EDH      10000.0      1.042690     0.313367   8450.614890   446.617150       31.735920       0.654970
       144   LEDH        200.0      0.969522     0.280392    163.129987    12.050006       13.212631       0.771987
       400    EDH        200.0      1.074939     0.279410    169.686958    16.080329        8.059855       0.403826
       400    EDH      10000.0      1.060482     0.274461   8720.020721   359.247661      168.876814       4.749912
       400   LEDH        200.0      0.968917     0.233755    171.050886    14.024257       96.086205       3.062138

✓ Detailed results saved to: /Users/amber_test/Desktop/filter/simulator/data/skewt_filt